# Connecting our Models

### Introduction

In this lesson, we'll see how we can use our objects and our orm functions to perform some of the SQL calls for us, and allow us to work with a list of object instances instead of database records.  Let's get started.

### Using an Object

Now so far, we have been successful at connecting a request to our API to a request to our database.  And we did so with code like the following:

```python
# application.py
import sqlite3
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/nhl/players/')
def show_players():
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players;')
    players = cursor.fetchall()
    return jsonify(players)

app.run(debug = True)
```

Now when we make a request to the `/nhl/players` url, each player is represented as a list.

<img src="./nhl_players_json.png" width="40%">

It would be better if our players were represented as dictionaries.  

Perhaps one way that we can get those list of dictionaries is to create a list of *player objects* with our return values from the database.  Remember, this is essentially what our `build_from_records` function did for us: initialized instances with records from our database.  Ok, let's see how we can do this.

> We can write our Player class in a `player.py` file.

In [12]:
class Player:
    __table__ = 'players'
    columns = ['id', 'name', 'age', 'height', 'weight', 
               'shot', 'birth_place', 'birthdate', 'number']
    
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

In [13]:
Player(name = 'claude giroux')

Then we can use `build_from_records` to convert our list of player records from the database into a list of player objects.

```python
@app.route('/nhl/players/')
def show_players():
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    cursor.execute('select * from players;')
    player_records = cursor.fetchall()
    players = build_from_records(Player, player_records)
    player_dicts = [player.__dict__ for player in players]
    return jsonify(player_dicts)
```

That's really it.  Our request to `/nhl/players` triggers a call to `SELECT * FROM players;`.  From this, we get a list of players `[[1, 'sean coutourier'], [2, 'kevin hayes']]`, and we pass this into our `build_from_players` function. 

### A last step

As a last step, remember that we have written a function in our orm to perform the proper `SELECT *` statement.  This is our `find_all` function.  Let's try using that here.

```python
@app.route('/nhl/players/')
def show_players():
    conn = sqlite3.connect('nhl.db')
    cursor = conn.cursor()
    players = find_all(Player,cursor)
    player_dicts = [player.__dict__ for player in players]
    return jsonify(player_dicts)
```

So we can see that we are able to use our find_all method to retrieve a list of player instances from the database.  Then we convert these instances to json and return them as our response.

### Summary

In this lesson, we moved further through different layers of abstraction to refactor our code.  We started with our code that returned each player represented as a list from our database.  Then we declared the class Player so that we turn our player records into player instances with the `build_from_records` function.  Finally, we used our `find_all` function from the orm that we previously wrote to make the SQL call for us.